### __Pipeline 2: SNP processing with imputation of the common allele__

#### __Requirements__
- a python environment (installed with conda for example);
- .TSV files from the obtained with pipeline 1;
- python `subprocess`;
- python `Numpy`;
- python `Pandas`;

Load some python libraries

In [1]:
import numpy as np
import pandas as pd
from snp_utils import impute_missing_haplotypes, filter_short_introns_from_bed, filter_snps_by_interval
from sfs_utils import create_unfolded_sfs_from_df, fold_sfs

#### __Import the tables into Pandas__

We are going to use pandas to import the SNP table. Pandas is a great (if used with caution) Python package built on Numpy which allows easy dataFrame manipulations.

In [2]:
%cd ../../masked/vcfs/tables/
!ls 

/Users/tur92196/DGN/dpgp3/masked/vcfs/tables
ZI_chr2L_ann_table.tsv ZI_chr3L_ann_table.tsv
ZI_chr2R_ann_table.tsv ZI_chr3R_ann_table.tsv


/opt/homebrew/anaconda3/envs/dmelnexus/lib/python3.11/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


Upload the `.tsv` file for each chromosome (except for the __chrom4__ and __chromX__).

In [3]:
# Upload files with pd.read_table()
chr2L_table = pd.read_table("ZI_chr2L_ann_table.tsv")
chr2R_table = pd.read_table("ZI_chr2R_ann_table.tsv")
chr3L_table = pd.read_table("ZI_chr3L_ann_table.tsv")
chr3R_table = pd.read_table("ZI_chr3R_ann_table.tsv")

Check the number of SNPs in each file:

In [4]:
# Take a look at the number of SNPs with .shape
"chr2L = {}, chr2R = {}, chr3L = {}, chr3R = {} SNPs!".format(chr2L_table.shape[0], chr2R_table.shape[0], chr3L_table.shape[0], chr3R_table.shape[0])

'chr2L = 2016260, chr2R = 1582937, chr3L = 1795061, chr3R = 2087683 SNPs!'

In [5]:
# Take a look at chr2R
chr2L_table.head()

,chrom,pos,id,ref,alt,refcount,altcount,refflank,altflank,refcodon,...,snpeff_trnscid,sift_trnscid,sift_geneid,sift_genename,sift_region,sift_vartype,sifts_core,sift_median,sift_pred,deleteriousness
0,chr2L,5090,.,T,C,103,3,CATTTTCTC,CATTCTCTC,NaN,...,FBtr0475186,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,chr2L,5095,.,T,A,16,104,TCTCTCCCA,TCTCACCCA,NaN,...,FBtr0475186,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,chr2L,5110,.,T,A,130,2,AGGGTGAAA,AGGGAGAAA,NaN,...,FBtr0475186,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,chr2L,5118,.,G,T,147,1,ATATGATCG,ATATTATCG,NaN,...,FBtr0475186,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,chr2L,5140,.,C,T,158,2,AGTGCCAAC,AGTGTCAAC,NaN,...,FBtr0475186,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### __Data imputation__

Check to see if all SNPs have data for at least 50% of the sample.
Then, identify the common alllele of each SNP, to impute to its count, the missing genotypes.

In [6]:
# Add a total count column to all datasets:
chr2L_table['totalcount'] = chr2L_table['refcount'] + chr2L_table['altcount']
chr2R_table['totalcount'] = chr2R_table['refcount'] + chr2R_table['altcount']
chr3L_table['totalcount'] = chr3L_table['refcount'] + chr3L_table['altcount']
chr3R_table['totalcount'] = chr3R_table['refcount'] + chr3R_table['altcount']

In [7]:
# Take a look at chr2R
chr2L_table.head()

,chrom,pos,id,ref,alt,refcount,altcount,refflank,altflank,refcodon,...,sift_trnscid,sift_geneid,sift_genename,sift_region,sift_vartype,sifts_core,sift_median,sift_pred,deleteriousness,totalcount
0,chr2L,5090,.,T,C,103,3,CATTTTCTC,CATTCTCTC,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,106
1,chr2L,5095,.,T,A,16,104,TCTCTCCCA,TCTCACCCA,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,120
2,chr2L,5110,.,T,A,130,2,AGGGTGAAA,AGGGAGAAA,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,132
3,chr2L,5118,.,G,T,147,1,ATATGATCG,ATATTATCG,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,148
4,chr2L,5140,.,C,T,158,2,AGTGCCAAC,AGTGTCAAC,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,160


Remove SNPs if the totalcount < 197/2. This should have no effect on this data, as I filtered out SNPs with proportion of missing genotypes > 50% (using vcftools).

In [8]:
chr2L_table = chr2L_table[chr2L_table['totalcount'] >= 197/2]
chr2R_table = chr2R_table[chr2R_table['totalcount'] >= 197/2]
chr3L_table = chr3L_table[chr3L_table['totalcount'] >= 197/2]
chr3R_table = chr3R_table[chr3R_table['totalcount'] >= 197/2]

In [9]:
# Take a look at the number of SNPs with .shape
"chr2L = {}, chr2R = {}, chr3L = {}, chr3R = {} SNPs!".format(chr2L_table.shape[0], chr2R_table.shape[0], chr3L_table.shape[0], chr3R_table.shape[0])

'chr2L = 2016260, chr2R = 1582937, chr3L = 1795061, chr3R = 2087683 SNPs!'

Imput missing data using the common allele

In [10]:
max_number_of_haplotypes = 197

In [16]:
# Impute missing haplotypes
chr2L_table_imputed = impute_missing_haplotypes(chr2L_table, max_number_of_haplotypes)
chr2R_table_imputed = impute_missing_haplotypes(chr2R_table, max_number_of_haplotypes)
chr3L_table_imputed = impute_missing_haplotypes(chr3L_table, max_number_of_haplotypes)
chr3R_table_imputed = impute_missing_haplotypes(chr3R_table, max_number_of_haplotypes)

#### __Subset SNPs__

__Subset each chromosome to retain only:__
- `SNPs in short-introns`;
- `Synonymous SNPs`;
- `Non-synonymous SNPs`;

__(1) Subset to retain only SNPs annotated as introns__

In [20]:

chr2L_introns = chr2L_table_imputed[(chr2L_table_imputed['effect'] == "INTRON")]
chr2R_introns = chr2R_table_imputed[(chr2R_table_imputed['effect'] == "INTRON")]
chr3L_introns = chr3L_table_imputed[(chr3L_table_imputed['effect'] == "INTRON")]
chr3R_introns = chr3R_table_imputed[(chr3R_table_imputed['effect'] == "INTRON")]


Do the same for the other chromosomes

In [21]:
# Take a look at the number of SNPs with .shape
"chr2L = {}, chr2R = {}, chr3L = {}, chr3R = {} SNPs!".format(chr2L_introns.shape[0], chr2R_introns.shape[0], chr3L_introns.shape[0], chr3R_introns.shape[0])

'chr2L = 372462, chr2R = 271472, chr3L = 337894, chr3R = 452918 SNPs!'

__(2)Keep only short intronic SNPs__

For each chromosome, retain only SNPs in `short introns`. To do that, you need to identify the short-intros in Dm6 genome and head/tail the sequence to remove head and trailing 8bp from each short-intron. These extremes migh be under selection constraints. Download the intron regions as `.BED` file from [here](https://genome.ucsc.edu/cgi-bin/hgTables). Select *D. melanogaster* assembly known as *Dm6* (Aug. 2014, BDGP Release 6 + ISO 1 MT/dm6). Define the region of interest as `genome`, select the output format as `BED` and name it. In the next page, select only `Introns plus 0`, then hit `Get BED`. Now you are ready to go. The next step is to create a Pandas DataFrame with short introns intervals.

In [48]:
%cd ../../../../reference/
!ls

/Users/tur92196/DGN/reference
dm3.fa                            dmel-2L-chromosome-r5.57.fasta.gz
dm3.fa.fai                        dmel-2L-r5.57.gff.gz
dm3ToDm6.over.chain               dmel-2R-chromosome-r5.57.fasta.gz
dm6.fa                            dmel-2R-r5.57.gff.gz
dm6.fa.dict                       dmel-3L-chromosome-r5.57.fasta.gz
dm6.fa.fai                        dmel-3L-r5.57.gff.gz
dm6_introns.bed                   dmel-3R-chromosome-r5.57.fasta.gz
dmel-2L-chromosome-r5.57.fasta    dmel-3R-r5.57.gff.gz


/opt/homebrew/anaconda3/envs/dmelnexus/lib/python3.11/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [49]:
short_introns = filter_short_introns_from_bed("dm6_introns.bed", 
                                              "dm6_short_introns.bed", 
                                              ["chr2L", "chr2R", "chr3L", "chr3R"],
                                              short_intron_size=86,
                                              trailling_size=8)
short_introns.head()

,0,1,2,3,4,5
0,chr2L,8387838,8387882,NM_001201797.2_intron_5_0_chr2L_8387832_f,0,+
1,chr2L,8387838,8387882,NM_001201795.2_intron_5_0_chr2L_8387832_f,0,+
2,chr2L,8387838,8387882,NM_164812.5_intron_4_0_chr2L_8387832_f,0,+
3,chr2L,8387838,8387882,NM_205936.3_intron_5_0_chr2L_8387832_f,0,+
4,chr2L,8387838,8387882,NM_205935.3_intron_5_0_chr2L_8387832_f,0,+


Now, apply the filter based on the BED file intervals (to make sure to only keep `short-intronic` SNPs)

In [52]:
# Keep only SNPs in exons and in short-introns
chr2L_short_introns = filter_snps_by_interval(chr2L_introns, short_introns)
chr2R_short_introns = filter_snps_by_interval(chr2R_introns, short_introns)
chr3L_short_introns = filter_snps_by_interval(chr3L_introns, short_introns)
chr3R_short_introns = filter_snps_by_interval(chr3R_introns, short_introns)

Check how many short-introns SNPs were retained:

In [53]:
# Take a look at the number of SNPs with .shape
"chr2L = {}, chr2R = {}, chr3L = {}, chr3R = {} SNPs!".format(chr2L_short_introns.shape[0], chr2R_short_introns.shape[0], chr3L_short_introns.shape[0], chr3R_short_introns.shape[0])

'chr2L = 2830, chr2R = 2463, chr3L = 2156, chr3R = 3057 SNPs!'

__(3) Take synonymous and non-synonymous SNPs__

In [58]:
chr2L_exons = chr2L_table_imputed[(chr2L_table_imputed['effect'] == "NON_SYNONYMOUS_CODING") | (chr2L_table_imputed['effect'] == "SYNONYMOUS_CODING")]
chr2R_exons = chr2R_table_imputed[(chr2R_table_imputed['effect'] == "NON_SYNONYMOUS_CODING") | (chr2R_table_imputed['effect'] == "SYNONYMOUS_CODING")]
chr3L_exons = chr3L_table_imputed[(chr3L_table_imputed['effect'] == "NON_SYNONYMOUS_CODING") | (chr3L_table_imputed['effect'] == "SYNONYMOUS_CODING")]
chr3R_exons = chr3R_table_imputed[(chr3R_table_imputed['effect'] == "NON_SYNONYMOUS_CODING") | (chr3R_table_imputed['effect'] == "SYNONYMOUS_CODING")]

Do the same on the other chromosomes

In [59]:
# Take a look at the number of SNPs with .shape
"chr2L = {}, chr2R = {}, chr3L = {}, chr3R = {} SNPs!".format(chr2L_exons.shape[0], chr2R_exons.shape[0], chr3L_exons.shape[0], chr3R_exons.shape[0])

'chr2L = 258554, chr2R = 238496, chr3L = 228226, chr3R = 272780 SNPs!'

Jody found some bug on SNPEff which make some SNPs annotated as synonymous or non-synomymous, but missing nucleotides on the codon string.
We might want to remove these SNPs. One simple solution is to look for SNPs where the codon string length is lower than 3.

In [93]:
codon_length_dict = {}
for i in chr2L_exons["refcodon"]:
    

3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3


#### __Get the SFSs__

For introns:

In [82]:
chr2L_short_introns_sfs = create_unfolded_sfs_from_df(chr2L_short_introns, "altcount", max_number_haplotypes)
chr2R_short_introns_sfs = create_unfolded_sfs_from_df(chr2R_short_introns, "altcount", max_number_haplotypes)
chr3L_short_introns_sfs = create_unfolded_sfs_from_df(chr3L_short_introns, "altcount", max_number_haplotypes)
chr3R_short_introns_sfs = create_unfolded_sfs_from_df(chr3R_short_introns, "altcount", max_number_haplotypes)

Then combined each chromosome short-introns SFS

In [83]:
short_introns_sfs_array = np.array([
    chr2L_short_introns_sfs,
    chr2R_short_introns_sfs,
    chr3L_short_introns_sfs,
    chr3R_short_introns_sfs
])

short_introns_sfs = np.sum(short_introns_sfs_array, 0).tolist()

For the synonymous SNPs

In [62]:
chr2L_synonymous = chr2L_exons[(chr2L_exons['effect'] == "SYNONYMOUS_CODING")]
chr2R_synonymous = chr2R_exons[(chr2R_exons['effect'] == "SYNONYMOUS_CODING")]
chr3L_synonymous = chr3L_exons[(chr3L_exons['effect'] == "SYNONYMOUS_CODING")]
chr3R_synonymous = chr3R_exons[(chr3R_exons['effect'] == "SYNONYMOUS_CODING")]

synonymous_sfs_array = np.array([
    create_unfolded_sfs_from_df(chr2L_synonymous, "altcount", max_number_haplotypes),
    create_unfolded_sfs_from_df(chr2R_synonymous, "altcount", max_number_haplotypes),
    create_unfolded_sfs_from_df(chr3L_synonymous, "altcount", max_number_haplotypes),
    create_unfolded_sfs_from_df(chr3R_synonymous, "altcount", max_number_haplotypes)
])

synonymous_sfs = np.sum(synonymous_sfs_array, 0).tolist()

And for non-synonymous ones:

In [63]:
chr2L_nonsynonymous = chr2L_exons[(chr2L_exons['effect'] == "NON_SYNONYMOUS_CODING")]
chr2R_nonsynonymous = chr2R_exons[(chr2R_exons['effect'] == "NON_SYNONYMOUS_CODING")]
chr3L_nonsynonymous = chr3L_exons[(chr3L_exons['effect'] == "NON_SYNONYMOUS_CODING")]
chr3R_nonsynonymous = chr3R_exons[(chr3R_exons['effect'] == "NON_SYNONYMOUS_CODING")]

nonsynonymous_sfs_array = np.array([
    create_unfolded_sfs_from_df(chr2L_nonsynonymous, "altcount", max_number_haplotypes),
    create_unfolded_sfs_from_df(chr2R_nonsynonymous, "altcount", max_number_haplotypes),
    create_unfolded_sfs_from_df(chr3L_nonsynonymous, "altcount", max_number_haplotypes),
    create_unfolded_sfs_from_df(chr3R_nonsynonymous, "altcount", max_number_haplotypes)
])

nonsynonymous_sfs = np.sum(nonsynonymous_sfs_array, 0).tolist()

#### __Fold the SFSs__

In [84]:
short_introns_sfs_folded = fold_sfs(short_introns_sfs)
synonymous_sfs_folded = fold_sfs(synonymous_sfs)
nonsynonymous_sfs_folded = fold_sfs(nonsynonymous_sfs)

Save the three SFSs to a file:

In [67]:
%cd ../dpgp3/masked/vcfs/sfss/
!ls 

/Users/tur92196/DGN/dpgp3/masked/vcfs/sfss
ZI_sfs_nopairing_all_introns_downsampled.txt
ZI_sfs_nopairing_downsampled.txt


/opt/homebrew/anaconda3/envs/dmelnexus/lib/python3.11/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [88]:
output_sfs_file = "ZI_sfs_si_nopairing_imputed_folded.txt"

with open(output_sfs_file, "w") as of:
    of.write("Introns, synonymous, and nonsynonymous SFS of " + str(max_number_haplotypes) + " samples" + "\n")
    of.write("\t".join(str(item) for item in short_introns_sfs_folded) + "\n")
    of.write("\n")
    of.write("\t".join(str(item) for item in synonymous_sfs_folded) + "\n")
    of.write("\n")
    of.write("\t".join(str(item) for item in nonsynonymous_sfs_folded) + "\n")